In [12]:
from os.path import isfile
import pandas as pd
from sanitizer import sanitize_and_save

In [15]:
if not isfile("../data/refined.parquet"):
    print("Local refined copy missing.")
    print("Proceed to load and process raw backup...")
    sanitize_and_save()
    print("Refined restoration complete.")
df = pd.read_parquet("../data/refined.parquet")